In [ ]:
import os
import joblib
import random
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
from keras.layers import Input, LSTM, Dense
from keras.models import Model
from keras.utils import to_categorical
from keras import Input
from tqdm.notebook import tqdm
import keras

In [ ]:
files = os.listdir("/content/drive/MyDrive/Video Captioning/Data/Training Videos/feat")
files = files[:300]
for i in range(len(files)):
  files[i] = files[i][:-8]
files

In [ ]:
# training_df = pd.read_csv("/content/drive/MyDrive/Video Captioning/Data/training_captions.csv")
# training_df['captions'] = training_df['captions'].str[:-1]
# training_df.head()

# training_df.to_csv("/content/drive/MyDrive/Video Captioning/Data/training_captions.csv")

In [ ]:
len(files)

300

In [ ]:
BASE_DIR = "/content/drive/MyDrive/Video Captioning/Data"
TRAIN_FEATURE_DIR = os.path.join(BASE_DIR,"Training Videos/feat")
features = {}
for file in files:
  f = np.load(os.path.join(TRAIN_FEATURE_DIR, file + ".avi.npy"), allow_pickle=True)
  features[file] = f

features

In [ ]:
len(features)

300

In [ ]:
with open(os.path.join(BASE_DIR, 'training_captions.csv'), 'r') as f:
    next(f)
    captions_doc = f.read()

In [ ]:
# create mapping of image to captions
mapping = {}
# process lines
for line in tqdm(captions_doc.split('\n')):
    #split the line by comma(,)
    tokens = line.split(',')
    if len(line) < 2:
        continue
    image_id, caption = tokens[0], tokens[1:]
    caption[0] = caption[0][1:]
    #remove extension from image id
    image_id = image_id.split('.')[0]
    # convert caption list to string
    caption = " ".join(caption)
    # create list if needed
    if image_id in files:
      if image_id not in mapping:
        mapping[image_id] = []
      mapping[image_id].append(caption)
mapping['9Q0JfdP36kI_56_64']

  0%|          | 0/153559 [00:00<?, ?it/s]

['a woman peels a small onion or garlic clove with her fingers and then cuts it with a knife',
 'a woman is peeling a clove a garlic',
 'a woman is peeling garlic',
 'a woman is peeling the outer skin off of a small onion',
 'a woman is peeling a garlic',
 'a woman is peeling a clove of garlic',
 'a woman is peeling a garlic',
 'a woman is peeling and mincing a clove of garlic',
 'a woman is peeling a garlic clove',
 'a woman is peeling some garlic',
 'a woman is peeling garlic',
 'the woman is peeling garlic',
 'a woman is peeling and crushing a clove of garlic',
 'a woman peels an onion',
 'a woman is peeling garlic',
 'the lady peeled the garlic',
 'she is doing the something',
 'doing chiken',
 'a woman is peeling garlic',
 'a woman peels garlic',
 'a woman is peeling garlic',
 'a lady making a dish in the kitchen',
 'a woman is peeling some garlic',
 'a japanes woman removing peel of garlic clove and mash it with single hands',
 'the lady peeled the skin off the garlic',
 'the lad

In [ ]:
mapping['9Q0JfdP36kI_56_64']

['a woman peels a small onion or garlic clove with her fingers and then cuts it with a knife',
 'a woman is peeling a clove a garlic',
 'a woman is peeling garlic',
 'a woman is peeling the outer skin off of a small onion',
 'a woman is peeling a garlic',
 'a woman is peeling a clove of garlic',
 'a woman is peeling a garlic',
 'a woman is peeling and mincing a clove of garlic',
 'a woman is peeling a garlic clove',
 'a woman is peeling some garlic',
 'a woman is peeling garlic',
 'the woman is peeling garlic',
 'a woman is peeling and crushing a clove of garlic',
 'a woman peels an onion',
 'a woman is peeling garlic',
 'the lady peeled the garlic',
 'she is doing the something',
 'doing chiken',
 'a woman is peeling garlic',
 'a woman peels garlic',
 'a woman is peeling garlic',
 'a lady making a dish in the kitchen',
 'a woman is peeling some garlic',
 'a japanes woman removing peel of garlic clove and mash it with single hands',
 'the lady peeled the skin off the garlic',
 'the lad

In [ ]:
def clean(mapping):
  for key, captions in mapping.items():
    for i in range(len(captions)):
      caption = captions[i]
      # preprocessing text
      # converting to lower case
      caption = caption.lower()
      # delete digits and special characters etc,
      caption = caption.replace('[A=Za-z]','')
      # delete addtional spaces
      caption = caption.replace('\s+', ' ')
      # add start and end tags to the caption
      caption = 'startseq ' + " ".join([word for word in caption.split() if len(word)>1])  + ' endseq'
      captions[i]= caption
    valid_captions = []
    # Removing captions that are smaller than 6 words or larger than 12
    for caption in captions:
      if len(caption.split(" ")) >= 6 and len(caption.split(" ")) <= 12:
        valid_captions.append(caption)

        # Assign the new valid captions list to the original captions list
    mapping[key] = valid_captions
clean(mapping)
mapping['9Q0JfdP36kI_56_64']

['startseq woman is peeling clove garlic endseq',
 'startseq woman is peeling garlic endseq',
 'startseq woman is peeling the outer skin off of small onion endseq',
 'startseq woman is peeling garlic endseq',
 'startseq woman is peeling clove of garlic endseq',
 'startseq woman is peeling garlic endseq',
 'startseq woman is peeling and mincing clove of garlic endseq',
 'startseq woman is peeling garlic clove endseq',
 'startseq woman is peeling some garlic endseq',
 'startseq woman is peeling garlic endseq',
 'startseq the woman is peeling garlic endseq',
 'startseq woman is peeling and crushing clove of garlic endseq',
 'startseq woman peels an onion endseq',
 'startseq woman is peeling garlic endseq',
 'startseq the lady peeled the garlic endseq',
 'startseq she is doing the something endseq',
 'startseq woman is peeling garlic endseq',
 'startseq woman is peeling garlic endseq',
 'startseq lady making dish in the kitchen endseq',
 'startseq woman is peeling some garlic endseq',
 'st

In [ ]:
mapping

In [ ]:
len(mapping)

293

In [ ]:
lengths = [len(v) for v in mapping.values()]
lengths

In [ ]:
sum(lengths)

10259

In [ ]:
train_path = "/content/drive/MyDrive/Video Captioning/Data/Testing Videos"
test_path = "/content/drive/MyDrive/Video Captioning/Data/Training Videos"
max_length = 12
batch_size = 10
learning_rate = 0.0007
epochs = 150
latent_dim = 512
num_encoder_tokens = 4096
num_decoder_tokens = 1500
time_steps_encoder = 80
time_steps_decoder = 12
max_probability = -1
save_model_path = "/content/drive/MyDrive/Video Captioning/model_final"
validation_split = 0.15
search_type = 'greedy'



all_captions = []
for key in mapping:
  for caption in mapping[key]:
      all_captions.append(caption)

In [ ]:
# get maximum length of the captions avilable
max_length = max(len(caption.split()) for caption in all_captions)
print(max_length)
min_length = min(len(caption.split()) for caption in all_captions)
print(min_length)

12
6


In [ ]:
# Train Test Split
image_ids = list(mapping.keys())
#print(image_ids)
random.shuffle(image_ids)
split = int(len(image_ids)*validation_split)
training_list = image_ids[split:]
validation_list = image_ids[:split]

In [ ]:
print(len(training_list))
print(len(validation_list))

250
43


In [ ]:
from collections import OrderedDict

In [ ]:
training_captions = []

for id in training_list:
  for caption in mapping[id]:
      training_captions.append(caption)

tokenizer = Tokenizer(num_words= num_decoder_tokens)
tokenizer.fit_on_texts(training_captions)

# if len(tokenizer.word_index) > num_decoder_tokens:
#     tokenizer.word_index = dict(list(tokenizer.word_index.items())[:num_decoder_tokens])
#     tokenizer.num_words = num_decoder_tokens

# my_dict = tokenizer.word_counts
# print(OrderedDict(sorted(my_dict.items(), key=lambda x: x[1], reverse=True)))

vocab_size =len(tokenizer.word_index)+1
# tokenizer.word_counts
vocab_size

3478

In [ ]:
def data_loader(training_list,mapping,features,tokenizer,max_length,epochs,num_decoder_tokens,batch_size):
  encoder_input_data = []
  decoder_input_data = []
  decoder_target_data = []
  videoId = []
  videoSeq = []
  for id in training_list:
    captions = mapping[id]
    for caption in captions:
      videoId.append(id)
      videoSeq.append(caption)

  train_sequences = tokenizer.texts_to_sequences(videoSeq)
  train_sequences = np.array(train_sequences, dtype=object)
  train_sequences = pad_sequences(train_sequences, padding='post', truncating='post',
                                  maxlen=max_length)

  file_size = len(train_sequences)
  n = 0
  for i in range(epochs):
      for idx in range(0, file_size):
          n += 1
          encoder_input_data.append(features[videoId[idx]])
          y = to_categorical(train_sequences[idx], num_decoder_tokens)
          decoder_input_data.append(y[:-1])
          decoder_target_data.append(y[1:])
          if n == batch_size or idx == file_size-1:
              encoder_input = np.array(encoder_input_data)
              decoder_input = np.array(decoder_input_data)
              decoder_target = np.array(decoder_target_data)
              encoder_input_data = []
              decoder_input_data = []
              decoder_target_data = []
              n = 0
              yield ([encoder_input, decoder_input], decoder_target)

In [ ]:
train_path = "/content/drive/MyDrive/Video Captioning/Data/Testing Videos"
test_path = "/content/drive/MyDrive/Video Captioning/Data/Training Videos"
max_length = 12
batch_size = 10
learning_rate = 0.0007
epochs = 150
latent_dim = 512
num_encoder_tokens = 4096
num_decoder_tokens = 1500
time_steps_encoder = 80
time_steps_decoder = 12
max_probability = -1
save_model_path = "/content/drive/MyDrive/Video Captioning/model_final"
validation_split = 0.15
search_type = 'greedy'

def train_model(training_list,validation_list):
  """
  an encoder decoder sequence to sequence model
  reference : https://arxiv.org/abs/1505.00487
  """
  encoder_inputs = Input(shape=(time_steps_encoder, num_encoder_tokens), name="encoder_inputs")
  encoder = LSTM(latent_dim, return_state=True, return_sequences=True, name='encoder_lstm')
  _, state_h, state_c = encoder(encoder_inputs)
  encoder_states = [state_h, state_c]

  decoder_inputs = Input(shape=(time_steps_decoder, num_decoder_tokens), name="decoder_inputs")
  decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')
  decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
  decoder_dense = Dense(num_decoder_tokens, activation='relu', name='decoder_relu')
  decoder_outputs = decoder_dense(decoder_outputs)

  model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
  model.summary()


  train = data_loader(training_list,mapping,features,tokenizer,max_length,epochs,num_decoder_tokens,batch_size)
  valid = data_loader(validation_list,mapping,features,tokenizer,max_length,epochs,num_decoder_tokens,batch_size)

  early_stopping = EarlyStopping(monitor='val_loss', patience=15, verbose=1, mode='min')

  # Run training
  opt = keras.optimizers.Adam(lr=0.0003)
  reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                factor=0.1, patience=5, verbose=0,
                                                mode="auto")
  model.compile(metrics=['accuracy'], optimizer=opt, loss='categorical_crossentropy')

  validation_steps = len(validation_list)//batch_size
  steps_per_epoch = len(training_list)//batch_size

  model.fit(train, validation_data=valid, validation_steps=validation_steps,
            epochs=epochs, steps_per_epoch=steps_per_epoch,
            callbacks=[reduce_lr, early_stopping])

  if not os.path.exists(save_model_path):
      os.makedirs(save_model_path)

  encoder_model = Model(encoder_inputs, encoder_states)
  decoder_state_input_h = Input(shape=(latent_dim,))
  decoder_state_input_c = Input(shape=(latent_dim,))
  decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
  decoder_outputs, state_h, state_c = decoder_lstm(
      decoder_inputs, initial_state=decoder_states_inputs)
  decoder_states = [state_h, state_c]
  decoder_outputs = decoder_dense(decoder_outputs)
  decoder_model = Model(
      [decoder_inputs] + decoder_states_inputs,
      [decoder_outputs] + decoder_states)
  #encoder_model.summary()
  #decoder_model.summary()

  # saving the models
  encoder_model.save(os.path.join(save_model_path, 'encoder_model.h5'))
  decoder_model.save_weights(os.path.join(save_model_path, 'decoder_model_weights.h5'))
  with open(os.path.join(save_model_path, 'tokenizer' + str(num_decoder_tokens)), 'wb') as file:
      joblib.dump(tokenizer, file)

In [ ]:
train_model(training_list,validation_list)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, 80, 4096)]   0           []                               
                                                                                                  
 decoder_inputs (InputLayer)    [(None, 12, 1500)]   0           []                               
                                                                                                  
 encoder_lstm (LSTM)            [(None, 80, 512),    9439232     ['encoder_inputs[0][0]']         
                                 (None, 512),                                                     
                                 (None, 512)]                                                     
                                                                                              

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/150
25/25 [==============================] - 9s 72ms/step - loss: 7.1637 - accuracy: 0.3400 - val_loss: 6.2852 - val_accuracy: 0.4364 - lr: 3.0000e-04
Epoch 2/150
25/25 [==============================] - 1s 37ms/step - loss: 6.9178 - accuracy: 0.4098 - val_loss: 6.0930 - val_accuracy: 0.4455 - lr: 3.0000e-04
Epoch 3/150
25/25 [==============================] - 1s 36ms/step - loss: 6.7996 - accuracy: 0.3978 - val_loss: 5.4782 - val_accuracy: 0.4977 - lr: 3.0000e-04
Epoch 4/150
25/25 [==============================] - 1s 37ms/step - loss: 6.9938 - accuracy: 0.3825 - val_loss: 6.5428 - val_accuracy: 0.4318 - lr: 3.0000e-04
Epoch 5/150
25/25 [==============================] - 1s 36ms/step - loss: 7.1523 - accuracy: 0.4007 - val_loss: 7.7845 - val_accuracy: 0.3500 - lr: 3.0000e-04
Epoch 6/150
25/25 [==============================] - 1s 36ms/step - loss: 6.0848 - accuracy: 0.4727 - val_loss: 6.2156 - val_accuracy: 0.4477 - lr: 3.0000e-04
Epoch 7/150
25/25 [===========================